In [ ]:
%pip install beautifulsoup4 pqdm pulp networkx

In [6]:
import pulp
import pandas as pd

In [7]:
winf_cbk = [
    "5105",  # Jahresabschluss und Unternehmensberichte
    "5107",  # Global Business
    "5106",  # Grundlagen der Wirtschaftsinformatik
    "5108",  # Funktionsübergreifende Betriebswirtschaftslehre - Prozesse und Entscheidungen
    "5056",  # Mikroökonomik (6056 ist Angewandte Mikroökonomik)
    "5059",  # Makroökonomik (6059 ist Internationale Makroökonomik)
    "5117",  # Zukunftsfähiges Wirtschaften: Vertiefung und Anwendung
    "5109",  # Wirtschaftsprivatrecht (6021 ist Wirtschaft im rechtlichen Kontext - Wirtschaftsprivatrecht I)
    "6023",  # Mathematik
    "6024",  # Statistik
    "5136",  # Standards wissenschaftlichen Arbeitens und Zitierens (6911 ist Grundlagen wissenschaftlichen Arbeitens)
]

# Vorraussetzungen um Kurse aus dem Hauptstudium zu belegen:
# mind 20 ECTS aus dem CBK

winf_hauptstudium = [
    "6012",  # Beschaffung, Logistik, Produktion
    "5155",  # Grundlagen und Methoden des Data und Knowledge Engineering
    "9485",  # Algorithmisches Denken und Programmierung
    "5158",  # Rechnernetzwerke und Datenübermittlung: Grundlagen und Sicherheit
    "5160",  # Design von betrieblichen Informationssystemen
    "5161",  # Governance und Management von IT-Projekten
    "5162",  # Forschungsmethoden der Wirtschaftsinformatik
]

vvzModel = pd.read_pickle("../0_daten/vvzModel.pkl")
relevantVvz = vvzModel[vvzModel["planpunkte_ids"].apply(lambda ids: any(str(id_) in (winf_cbk + winf_hauptstudium) for id_ in ids))]

In [8]:
import pulp

# Linear programming implementation to maximize ECTS

def get_lp_pick(vvz_df, cbk_ids, hs_ids, max_overlap_minutes=15):
    # Preprocess: filter out courses on Monday or Tuesday
    def is_on_valid_day(dates):
        for s in dates:
            # if s['start'].strftime('%A') in ['Monday', 'Tuesday']:
            if s['start'].strftime('%A') in ['Monday']:
                return False
        return True
    
    df = vvz_df.copy()
    df['valid_day'] = df['dates'].apply(is_on_valid_day)
    df = df[df['valid_day']].reset_index(drop=True)
    df['first_start'] = df['dates'].apply(lambda d: min(s['start'] for s in d))
    df['last_end']   = df['dates'].apply(lambda d: max(s['end']   for s in d))

    # Identify CBK and HS courses indices
    courses = list(df.index)
    cbk_set = set(courses[i] for i,iids in enumerate(df['planpunkte_ids']) if any(pid in cbk_ids for pid in iids))
    hs_set = set(courses[i] for i,iids in enumerate(df['planpunkte_ids']) if any(pid in hs_ids for pid in iids))

    # Compute conflict pairs (overlap > max_overlap_minutes)
    def overlaps_too_much(d1, d2):
        for s1 in d1:
            for s2 in d2:
                latest_start = max(s1['start'], s2['start'])
                earliest_end = min(s1['end'], s2['end'])
                if (earliest_end - latest_start).total_seconds() / 60 > max_overlap_minutes:
                    return True
        return False

    conflict_pairs = []
    for i in courses:
        for j in courses:
            if j <= i: continue
            if overlaps_too_much(df.at[i, 'dates'], df.at[j, 'dates']):
                conflict_pairs.append((i, j))

    # Planpunkt uniqueness: map planpunkt -> courses
    planpunkt_map = {}
    for i, pids in zip(courses, df['planpunkte_ids']):
        for pid in pids:
            planpunkt_map.setdefault(pid, []).append(i)

    # Instantiate LP problem
    prob = pulp.LpProblem("Course_Selection", pulp.LpMaximize)

    # Decision variables for each course
    x = pulp.LpVariable.dicts('x', courses, lowBound=0, upBound=1, cat='Binary')

    # Auxiliary binary: allow HS only if CBK ECTS >= 20
    z = pulp.LpVariable('z_cbk_ready', lowBound=0, upBound=1, cat='Binary')

    # Objective: maximize total ECTS
    prob += pulp.lpSum(df.at[i, 'ects'] * x[i] for i in courses)

    # CBK ECTS constraint
    prob += pulp.lpSum(df.at[i, 'ects'] * x[i] for i in cbk_set) >= 20 * z

    # HS selection only if z == 1
    for j in hs_set:
        prob += x[j] <= z

    for j in hs_set:
        prob += (
            pulp.lpSum(
                df.at[i, 'ects'] * x[i]
                for i in cbk_set
                if df.at[i, 'last_end'] <= df.at[j, 'first_start']
            )
            >= 0 * x[j]
        )

    # Conflict constraints
    for i, j in conflict_pairs:
        prob += x[i] + x[j] <= 1

    # Planpunkt uniqueness
    for pid, idxs in planpunkt_map.items():
        prob += pulp.lpSum(x[i] for i in idxs) <= 1

    # Solve silently
    prob.solve(pulp.PULP_CBC_CMD(msg=False))

    # Collect selected courses
    selected = [i for i in courses if pulp.value(x[i]) == 1]
    return (df.loc[selected], prob, conflict_pairs)

(selected_courses_lp, prob, conflict_pairs) = get_lp_pick(relevantVvz, winf_cbk, winf_hauptstudium)
selected_courses_lp["ects"].sum()

np.float64(58.0)

In [9]:
def print_lp_structure(prob):
    print("Objective function:")
    print(prob.objective)
    print("\nConstraints:")
    for name, constraint in prob.constraints.items():
        print(f"{name}: {constraint}")
print_lp_structure(prob)

Objective function:
4.0*x_0 + 4.0*x_1 + 3.0*x_10 + 4.0*x_100 + 4.0*x_101 + 4.0*x_102 + 4.0*x_103 + 4.0*x_104 + 4.0*x_105 + 4.0*x_106 + 4.0*x_107 + 4.0*x_108 + 8.0*x_109 + 3.0*x_11 + 4.0*x_110 + 4.0*x_111 + 4.0*x_112 + 4.0*x_113 + 4.0*x_114 + 8.0*x_115 + 6.0*x_116 + 4.0*x_117 + 4.0*x_118 + 3.0*x_119 + 4.0*x_12 + 3.0*x_120 + 3.0*x_121 + 3.0*x_122 + 3.0*x_123 + 3.0*x_124 + 3.0*x_125 + 3.0*x_126 + 3.0*x_127 + 4.0*x_13 + 4.0*x_14 + 3.0*x_15 + 4.0*x_16 + 4.0*x_17 + 3.0*x_18 + 4.0*x_19 + 4.0*x_2 + 4.0*x_20 + 4.0*x_21 + 4.0*x_22 + 4.0*x_23 + 3.0*x_24 + 4.0*x_25 + 4.0*x_26 + 3.0*x_27 + 4.0*x_28 + 4.0*x_29 + 4.0*x_3 + 4.0*x_30 + 4.0*x_31 + 4.0*x_32 + 4.0*x_33 + 4.0*x_34 + 4.0*x_35 + 4.0*x_36 + 4.0*x_37 + 3.0*x_38 + 4.0*x_39 + 3.0*x_4 + 3.0*x_40 + 4.0*x_41 + 3.0*x_42 + 3.0*x_43 + 4.0*x_44 + 4.0*x_45 + 4.0*x_46 + 4.0*x_47 + 4.0*x_48 + 4.0*x_49 + 3.0*x_5 + 4.0*x_50 + 4.0*x_51 + 4.0*x_52 + 4.0*x_53 + 4.0*x_54 + 4.0*x_55 + 4.0*x_56 + 4.0*x_57 + 4.0*x_58 + 4.0*x_59 + 3.0*x_6 + 4.0*x_60 + 4.0*x_61 + 4.